In [ ]:
#@title 🔧 **Environment Setup & Repository Clone**
import os
import sys
import subprocess
import shutil
from pathlib import Path
import importlib

# Configuration
REPO_URL = "https://github.com/remphanostar/SD-LongNose.git"
REPO_DIR = "/content/SD-LongNose"
MODULE_PATH = "/content/SD-LongNose/github_upload"

print("🚀 PINOKIO CLOUD GPU - COMPLETE SETUP")
print("=" * 60)

# Remove existing repo
if os.path.exists(REPO_DIR):
    print("🗑️ Removing existing repository...")
    shutil.rmtree(REPO_DIR)

# Clone fresh repo
print("📥 Cloning repository from GitHub...")
try:
    result = subprocess.run([
        "git", "clone", "--depth", "1", REPO_URL, REPO_DIR
    ], capture_output=True, text=True, check=True)
    print("✅ Repository cloned successfully")
except subprocess.CalledProcessError as e:
    print(f"❌ Git clone failed: {e}")
    print("stderr:", e.stderr)
    raise

# Add modules to Python path
if MODULE_PATH not in sys.path:
    sys.path.insert(0, MODULE_PATH)
    print(f"✅ Added to Python path: {MODULE_PATH}")

# Verify files exist
required_files = [
    "pinokio_cloud_main.py",
    "modules/pinokio_controller.py", 
    "modules/pinokio_installer.py",
    "modules/platform_detector.py",
    "modules/tunnel_manager.py"
]

print("\n🔍 Verifying repository files...")
for file in required_files:
    file_path = Path(MODULE_PATH) / file
    if file_path.exists():
        size = file_path.stat().st_size
        print(f"  ✅ {file} ({size:,} bytes)")
    else:
        print(f"  ❌ {file} - MISSING!")
        raise FileNotFoundError(f"Required file missing: {file}")

print("\n✅ Environment setup complete!")
print("📂 Repository:", REPO_DIR)
print("🐍 Module path:", MODULE_PATH)


In [ ]:
#@title 📦 **Install Required Dependencies**
import subprocess
import sys

def install_package(package, description=""):
    """Install a package with better error handling"""
    try:
        print(f"📦 Installing {package}...")
        result = subprocess.run([
            sys.executable, "-m", "pip", "install", "-q", package
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"  ✅ {package} installed successfully")
        else:
            print(f"  ❌ Failed to install {package}")
            print(f"  Error: {result.stderr}")
    except Exception as e:
        print(f"  ❌ Exception installing {package}: {e}")

# Required packages
packages = {
    "ipywidgets>=8.0.0": "Interactive widgets",
    "requests": "HTTP requests",
    "psutil": "System monitoring", 
    "pyngrok": "ngrok tunneling",
    "cloud-detect": "Cloud platform detection"
}

print("📦 INSTALLING DEPENDENCIES")
print("=" * 40)

for package, desc in packages.items():
    install_package(package, desc)

# Install additional system packages
print("\n🔧 Installing system packages...")
system_packages = ["wget", "curl", "git", "xvfb", "nodejs", "npm"]

for pkg in system_packages:
    try:
        subprocess.run(["apt-get", "install", "-y", "-q", pkg], 
                      capture_output=True, check=False)
    except:
        pass

print("\n✅ Dependencies installation complete!")


In [ ]:
#@title 🔌 **Import Real Modules from GitHub**
import importlib
import sys

# Clear any cached modules to ensure fresh imports
modules_to_clear = [
    "pinokio_cloud_main", "pinokio_controller", "pinokio_installer",
    "platform_detector", "tunnel_manager"
]

for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]

print("🔌 IMPORTING REAL MODULES")
print("=" * 40)

try:
    # Import the main orchestrator
    from pinokio_cloud_main import PinokioCloudGPU, quick_start_notebook
    print("✅ PinokioCloudGPU imported successfully")
    
    # Import individual modules for verification
    from modules.pinokio_controller import PinokioController
    print("✅ PinokioController imported successfully")
    
    from modules.pinokio_installer import PinokioInstaller  
    print("✅ PinokioInstaller imported successfully")
    
    from modules.platform_detector import PlatformDetector
    print("✅ PlatformDetector imported successfully")
    
    from modules.tunnel_manager import TunnelManager
    print("✅ TunnelManager imported successfully")
    
    print("\n🎉 All modules imported successfully from GitHub!")
    print("📁 Source: https://github.com/remphanostar/SD-LongNose")
    
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("\n🔍 Debugging info:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path includes: {MODULE_PATH}")
    print(f"Module path exists: {os.path.exists(MODULE_PATH)}")
    
    # List available files
    if os.path.exists(MODULE_PATH):
        print("\nAvailable files:")
        for file in os.listdir(MODULE_PATH):
            print(f"  - {file}")
    raise


In [ ]:
#@title 🎨 **Advanced Control Panel UI**
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import threading
import time
import json

# Global variables for UI state
pinokio_instance = None
current_status = {}
log_output = []

# Create UI components
class PinokioUI:
    def __init__(self):
        self.setup_styles()
        self.create_widgets()
        self.create_layout()
        
    def setup_styles(self):
        """Setup custom CSS styles"""
        self.style = """
        <style>
        .pinokio-header {
            background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 20px;
            border-radius: 10px;
            text-align: center;
            margin-bottom: 20px;
            font-family: 'Arial', sans-serif;
        }
        .pinokio-section {
            background: #f8f9fa;
            border: 1px solid #dee2e6;
            border-radius: 8px;
            padding: 15px;
            margin: 10px 0;
        }
        .status-running { color: #28a745; font-weight: bold; }
        .status-stopped { color: #dc3545; font-weight: bold; }
        .status-installing { color: #ffc107; font-weight: bold; }
        </style>
        """
    
    def create_widgets(self):
        """Create all UI widgets"""
        
        # Status display
        self.status_html = widgets.HTML(
            value="<div class='status-stopped'>● Stopped</div>",
            description=""
        )
        
        # Control buttons
        self.setup_btn = widgets.Button(
            description="🔧 Setup Environment",
            button_style='primary',
            layout=widgets.Layout(width='200px', height='40px')
        )
        
        self.install_btn = widgets.Button(
            description="📦 Install Pinokio", 
            button_style='info',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=True
        )
        
        self.start_btn = widgets.Button(
            description="🚀 Start Server",
            button_style='success', 
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=True
        )
        
        self.tunnel_btn = widgets.Button(
            description="🌐 Setup Tunnel",
            button_style='warning',
            layout=widgets.Layout(width='200px', height='40px'), 
            disabled=True
        )
        
        self.stop_btn = widgets.Button(
            description="⏹️ Stop All",
            button_style='danger',
            layout=widgets.Layout(width='200px', height='40px'),
            disabled=True
        )
        
        # Configuration options
        self.port_input = widgets.IntText(
            value=42000,
            description="Port:",
            style={'description_width': 'initial'}
        )
        
        self.tunnel_service = widgets.Dropdown(
            options=[('Auto (Recommended)', 'auto'), ('Cloudflare', 'cloudflare'), 
                    ('ngrok', 'ngrok'), ('LocalTunnel', 'localtunnel')],
            value='auto',
            description="Tunnel Service:",
            style={'description_width': 'initial'}
        )
        
        self.ngrok_token = widgets.Text(
            placeholder="Optional ngrok token",
            description="ngrok Token:",
            style={'description_width': 'initial'}
        )
        
        # Tool installation
        self.tool_dropdown = widgets.Dropdown(
            options=[
                ('Automatic1111 WebUI', 'automatic1111'),
                ('ComfyUI', 'comfyui'), 
                ('Text Generation WebUI', 'text-generation'),
                ('Fooocus', 'fooocus'),
                ('InvokeAI', 'invokeai')
            ],
            description="AI Tool:",
            style={'description_width': 'initial'}
        )
        
        self.install_tool_btn = widgets.Button(
            description="🤖 Install Tool",
            button_style='info',
            layout=widgets.Layout(width='150px'),
            disabled=True
        )
        
        # Progress and logs
        self.progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description="Progress:",
            bar_style='',
            style={'bar_color': '#1f77b4'},
            layout=widgets.Layout(width='100%')
        )
        
        self.log_output = widgets.Output(
            layout=widgets.Layout(
                height='300px', 
                border='1px solid #ccc',
                overflow_y='auto'
            )
        )
        
        # Status info
        self.info_html = widgets.HTML(
            value="<div>Ready to start...</div>"
        )
        
        # URLs display
        self.urls_html = widgets.HTML(
            value="<div>No active URLs</div>"
        )
        
        # Bind events
        self.setup_btn.on_click(self.on_setup_click)
        self.install_btn.on_click(self.on_install_click)
        self.start_btn.on_click(self.on_start_click)
        self.tunnel_btn.on_click(self.on_tunnel_click)
        self.stop_btn.on_click(self.on_stop_click)
        self.install_tool_btn.on_click(self.on_install_tool_click)
    
    def create_layout(self):
        """Create the main layout"""
        
        header = widgets.HTML(
            value="""
            <div class='pinokio-header'>
                <h1>🚀 Pinokio Cloud GPU</h1>
                <p>Complete AI Platform Deployment</p>
                <p><small>Using Real GitHub Implementation</small></p>
            </div>
            """ + self.style
        )
        
        # Main controls section
        controls_section = widgets.VBox([
            widgets.HTML("<h3>🎛️ Main Controls</h3>"),
            widgets.HBox([self.status_html, self.info_html]),
            self.progress,
            widgets.HBox([
                self.setup_btn, self.install_btn, self.start_btn, 
                self.tunnel_btn, self.stop_btn
            ], layout=widgets.Layout(flex_flow='row wrap')),
        ])
        
        # Configuration section
        config_section = widgets.VBox([
            widgets.HTML("<h3>⚙️ Configuration</h3>"),
            widgets.HBox([self.port_input, self.tunnel_service]),
            self.ngrok_token
        ])
        
        # Tools section  
        tools_section = widgets.VBox([
            widgets.HTML("<h3>🤖 AI Tools</h3>"),
            widgets.HBox([self.tool_dropdown, self.install_tool_btn])
        ])
        
        # Status section
        status_section = widgets.VBox([
            widgets.HTML("<h3>🌐 Access URLs</h3>"),
            self.urls_html
        ])
        
        # Logs section
        logs_section = widgets.VBox([
            widgets.HTML("<h3>📋 System Logs</h3>"),
            self.log_output
        ])
        
        # Combine all sections
        self.main_ui = widgets.VBox([
            header,
            widgets.HBox([
                widgets.VBox([controls_section, config_section], 
                           layout=widgets.Layout(width='48%')),
                widgets.VBox([tools_section, status_section], 
                           layout=widgets.Layout(width='48%'))
            ]),
            logs_section
        ])
    
    def log(self, message, level="INFO"):
        """Add message to log output"""
        timestamp = time.strftime("%H:%M:%S")
        with self.log_output:
            if level == "ERROR":
                print(f"🔴 [{timestamp}] {message}")
            elif level == "SUCCESS":
                print(f"🟢 [{timestamp}] {message}")
            elif level == "WARNING":
                print(f"🟡 [{timestamp}] {message}")
            else:
                print(f"🔵 [{timestamp}] {message}")
    
    def update_status(self, status, info=""):
        """Update status display"""
        if status == "running":
            self.status_html.value = "<div class='status-running'>● Running</div>"
        elif status == "installing":
            self.status_html.value = "<div class='status-installing'>● Installing</div>"
        elif status == "setting_up":
            self.status_html.value = "<div class='status-installing'>● Setting Up</div>"
        else:
            self.status_html.value = "<div class='status-stopped'>● Stopped</div>"
        
        self.info_html.value = f"<div><small>{info}</small></div>"
    
    def update_progress(self, value, description=""):
        """Update progress bar"""
        self.progress.value = value
        if description:
            self.progress.description = description
    
    def update_urls(self, urls):
        """Update URLs display"""
        if urls:
            html = "<div>"
            for name, url in urls.items():
                html += f"<p><strong>{name}:</strong> <a href='{url}' target='_blank'>{url}</a></p>"
            html += "</div>"
            self.urls_html.value = html
        else:
            self.urls_html.value = "<div>No active URLs</div>"
    
    def enable_buttons(self, buttons):
        """Enable specific buttons"""
        all_buttons = [self.setup_btn, self.install_btn, self.start_btn, 
                      self.tunnel_btn, self.stop_btn, self.install_tool_btn]
        
        for btn in all_buttons:
            btn.disabled = btn not in buttons
    
    # Event handlers
    def on_setup_click(self, btn):
        """Handle setup button click"""
        global pinokio_instance
        
        self.log("🔧 Starting environment setup...")
        self.update_status("setting_up", "Detecting platform...")
        self.update_progress(0, "Setup:")
        self.enable_buttons([])
        
        def setup_thread():
            global pinokio_instance
            try:
                # Initialize Pinokio instance
                pinokio_instance = PinokioCloudGPU(log_level="INFO")
                self.update_progress(25, "Setup:")
                
                # Setup environment
                success = pinokio_instance.setup()
                self.update_progress(100, "Setup:")
                
                if success:
                    self.log("✅ Environment setup complete!", "SUCCESS")
                    self.update_status("stopped", "Ready to install Pinokio")
                    self.enable_buttons([self.install_btn, self.stop_btn])
                else:
                    self.log("❌ Environment setup failed", "ERROR")
                    self.update_status("stopped", "Setup failed")
                    self.enable_buttons([self.setup_btn])
                    
            except Exception as e:
                self.log(f"❌ Setup error: {str(e)}", "ERROR")
                self.update_status("stopped", "Setup failed")
                self.enable_buttons([self.setup_btn])
        
        threading.Thread(target=setup_thread, daemon=True).start()
    
    def on_install_click(self, btn):
        """Handle install button click"""
        self.log("📦 Installing Pinokio...")
        self.update_status("installing", "Downloading and installing...")
        self.update_progress(0, "Install:")
        self.enable_buttons([])
        
        def install_thread():
            try:
                success = pinokio_instance.install_pinokio()
                self.update_progress(100, "Install:")
                
                if success:
                    self.log("✅ Pinokio installation complete!", "SUCCESS")
                    self.update_status("stopped", "Ready to start server")
                    self.enable_buttons([self.start_btn, self.stop_btn])
                else:
                    self.log("❌ Pinokio installation failed", "ERROR")
                    self.update_status("stopped", "Installation failed")
                    self.enable_buttons([self.install_btn, self.stop_btn])
                    
            except Exception as e:
                self.log(f"❌ Installation error: {str(e)}", "ERROR")
                self.update_status("stopped", "Installation failed")
                self.enable_buttons([self.install_btn, self.stop_btn])
        
        threading.Thread(target=install_thread, daemon=True).start()
    
    def on_start_click(self, btn):
        """Handle start button click"""
        port = self.port_input.value
        self.log(f"🚀 Starting Pinokio server on port {port}...")
        self.update_status("setting_up", f"Starting server on port {port}")
        self.update_progress(0, "Starting:")
        self.enable_buttons([])
        
        def start_thread():
            try:
                success = pinokio_instance.start_pinokio(port=port)
                self.update_progress(100, "Starting:")
                
                if success:
                    self.log("✅ Pinokio server started successfully!", "SUCCESS")
                    self.update_status("running", f"Server running on port {port}")
                    self.enable_buttons([self.tunnel_btn, self.install_tool_btn, self.stop_btn])
                    
                    # Update URLs with local access
                    self.update_urls({"Local Server": f"http://localhost:{port}"})
                else:
                    self.log("❌ Failed to start Pinokio server", "ERROR")
                    self.update_status("stopped", "Server start failed")
                    self.enable_buttons([self.start_btn, self.stop_btn])
                    
            except Exception as e:
                self.log(f"❌ Server start error: {str(e)}", "ERROR")
                self.update_status("stopped", "Server start failed")
                self.enable_buttons([self.start_btn, self.stop_btn])
        
        threading.Thread(target=start_thread, daemon=True).start()
    
    def on_tunnel_click(self, btn):
        """Handle tunnel button click"""
        service = self.tunnel_service.value
        ngrok_token = self.ngrok_token.value or None
        
        self.log(f"🌐 Setting up {service} tunnel...")
        self.update_status("running", f"Setting up {service} tunnel...")
        self.update_progress(0, "Tunnel:")
        self.enable_buttons([])
        
        def tunnel_thread():
            try:
                tunnel_url = pinokio_instance.setup_tunnel(
                    service=service,
                    ngrok_token=ngrok_token
                )
                self.update_progress(100, "Tunnel:")
                
                if tunnel_url:
                    self.log(f"✅ Tunnel ready: {tunnel_url}", "SUCCESS")
                    self.update_status("running", "Server + Tunnel active")
                    self.enable_buttons([self.install_tool_btn, self.stop_btn])
                    
                    # Update URLs
                    urls = {
                        "Public URL": tunnel_url,
                        "Local Server": f"http://localhost:{pinokio_instance.status.server_port}"
                    }
                    self.update_urls(urls)
                else:
                    self.log("⚠️ Tunnel setup failed, using local access", "WARNING")
                    self.update_status("running", "Server running (local only)")
                    self.enable_buttons([self.tunnel_btn, self.install_tool_btn, self.stop_btn])
                    
            except Exception as e:
                self.log(f"❌ Tunnel error: {str(e)}", "ERROR")
                self.update_status("running", "Server running (tunnel failed)")
                self.enable_buttons([self.tunnel_btn, self.install_tool_btn, self.stop_btn])
        
        threading.Thread(target=tunnel_thread, daemon=True).start()
    
    def on_install_tool_click(self, btn):
        """Handle tool installation"""
        tool = self.tool_dropdown.value
        
        self.log(f"🤖 Installing {tool}...")
        self.update_progress(0, "Tool:")
        self.enable_buttons([])
        
        def tool_thread():
            try:
                tool_url = pinokio_instance.install_ai_tool(tool)
                self.update_progress(100, "Tool:")
                
                if tool_url:
                    self.log(f"✅ {tool} installed and running: {tool_url}", "SUCCESS")
                    # Update URLs to include tool
                    current_urls = {}
                    if pinokio_instance.status.tunnel_url:
                        current_urls["Pinokio"] = pinokio_instance.status.tunnel_url
                    current_urls[f"{tool.title()}"] = tool_url
                    self.update_urls(current_urls)
                else:
                    self.log(f"❌ Failed to install {tool}", "ERROR")
                
                self.enable_buttons([self.install_tool_btn, self.stop_btn])
                
            except Exception as e:
                self.log(f"❌ Tool installation error: {str(e)}", "ERROR")
                self.enable_buttons([self.install_tool_btn, self.stop_btn])
        
        threading.Thread(target=tool_thread, daemon=True).start()
    
    def on_stop_click(self, btn):
        """Handle stop button click"""
        self.log("⏹️ Stopping all services...")
        self.update_status("stopped", "Shutting down...")
        self.update_progress(0, "Stopping:")
        self.enable_buttons([])
        
        def stop_thread():
            try:
                if pinokio_instance:
                    pinokio_instance.cleanup()
                    self.update_progress(100, "Stopping:")
                    self.log("✅ All services stopped", "SUCCESS")
                    self.update_status("stopped", "All services stopped")
                    self.update_urls({})
                    self.enable_buttons([self.setup_btn])
                else:
                    self.log("ℹ️ No services were running", "INFO")
                    self.enable_buttons([self.setup_btn])
                    
            except Exception as e:
                self.log(f"⚠️ Cleanup error: {str(e)}", "WARNING")
                self.enable_buttons([self.setup_btn])
        
        threading.Thread(target=stop_thread, daemon=True).start()
    
    def display(self):
        """Display the UI"""
        display(self.main_ui)

# Initialize and display UI
print("🎨 Creating Advanced UI...")
ui = PinokioUI()
ui.display()
print("✅ UI Ready! Use the controls above to manage Pinokio.")


In [ ]:
#@title 🚀 **Quick Start (Alternative to UI)**
def quick_start():
    """Quick start function for users who prefer code over UI"""
    print("🚀 PINOKIO QUICK START")
    print("=" * 40)
    
    try:
        # Use the real GitHub implementation
        pinokio, tunnel_url = quick_start_notebook()
        
        if pinokio and tunnel_url:
            print(f"\n🎉 SUCCESS!")
            print(f"🌐 Pinokio is accessible at: {tunnel_url}")
            print(f"📱 Open this URL in your browser")
            return pinokio, tunnel_url
        else:
            print("❌ Quick start failed")
            return None, None
            
    except Exception as e:
        print(f"❌ Quick start error: {e}")
        return None, None

# Uncomment to use quick start instead of UI:
# pinokio, url = quick_start()

In [ ]:
#@title 📊 **Advanced Status & Monitoring**
import ipywidgets as widgets
from IPython.display import display
import json
import time
import threading

def create_status_monitor():
    """Create a real-time status monitor"""
    
    status_output = widgets.Output(
        layout=widgets.Layout(height='400px', border='1px solid #ccc')
    )
    
    refresh_btn = widgets.Button(
        description="🔄 Refresh Status",
        button_style='info'
    )
    
    auto_refresh = widgets.Checkbox(
        value=False,
        description="Auto-refresh (5s)",
        style={'description_width': 'initial'}
    )
    
    def update_status():
        """Update the status display"""
        with status_output:
            status_output.clear_output(wait=True)
            
            if pinokio_instance:
                try:
                    status = pinokio_instance.get_status()
                    
                    print("📊 PINOKIO STATUS REPORT")
                    print("=" * 50)
                    
                    # Deployment status
                    deploy = status['deployment']
                    print(f"🚀 State: {deploy['state']}")
                    print(f"🖥️ Platform: {deploy['platform']}")
                    if deploy['started_at']:
                        print(f"⏰ Started: {deploy['started_at']}")
                    if deploy['last_error']:
                        print(f"❌ Last Error: {deploy['last_error']}")
                    
                    # Pinokio status  
                    pinokio_status = status['pinokio']
                    running = "✅ Running" if pinokio_status['running'] else "❌ Stopped"
                    print(f"\n📦 Pinokio: {running}")
                    if pinokio_status['port']:
                        print(f"🔌 Port: {pinokio_status['port']}")
                    if pinokio_status['version']:
                        print(f"📝 Version: {pinokio_status['version']}")
                    
                    # Tunnel status
                    tunnel = status['tunnel']  
                    if tunnel['url']:
                        print(f"\n🌐 Public URL: {tunnel['url']}")
                        print(f"🚇 Service: {tunnel['service']}")
                    else:
                        print(f"\n🌐 Tunnel: Not configured")
                    
                    # Tools
                    tools = status['tools']
                    print(f"\n🤖 Installed Tools: {tools['count']}")
                    for tool in tools['installed']:
                        print(f"  • {tool}")
                    
                    # System info
                    system = status['system']
                    print(f"\n💻 Platform: {system['platform']}")
                    if system['paths']:
                        print(f"📁 Pinokio Path: {system['paths'].get('pinokio', 'N/A')}")
                    
                    print("\n" + "=" * 50)
                    
                except Exception as e:
                    print(f"❌ Error getting status: {e}")
            else:
                print("ℹ️ Pinokio not initialized")
                print("Use the UI above to start Pinokio")
    
    def on_refresh_click(btn):
        update_status()
    
    def auto_refresh_loop():
        while auto_refresh.value:
            update_status()
            time.sleep(5)
    
    def on_auto_refresh_change(change):
        if change['new']:  # Auto-refresh enabled
            threading.Thread(target=auto_refresh_loop, daemon=True).start()
    
    refresh_btn.on_click(on_refresh_click)
    auto_refresh.observe(on_auto_refresh_change, names='value')
    
    # Initial status update
    update_status()
    
    # Display monitor
    monitor_ui = widgets.VBox([
        widgets.HTML("<h3>📊 System Status Monitor</h3>"),
        widgets.HBox([refresh_btn, auto_refresh]),
        status_output
    ])
    
    return monitor_ui

# Create and display the status monitor
status_monitor = create_status_monitor()
display(status_monitor)

In [ ]:
#@title 🔧 **Troubleshooting & Debug Tools**
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os

def create_debug_tools():
    """Create debugging and troubleshooting tools"""
    
    debug_output = widgets.Output(
        layout=widgets.Layout(height='300px', border='1px solid #ccc')
    )
    
    # Debug buttons
    check_env_btn = widgets.Button(
        description="🔍 Check Environment",
        button_style='info'
    )
    
    check_processes_btn = widgets.Button(
        description="🔍 Check Processes", 
        button_style='info'
    )
    
    check_ports_btn = widgets.Button(
        description="🔍 Check Ports",
        button_style='info'
    )
    
    clear_logs_btn = widgets.Button(
        description="🗑️ Clear Logs",
        button_style='warning'
    )
    
    def check_environment():
        """Check system environment"""
        with debug_output:
            print("🔍 ENVIRONMENT CHECK")
            print("=" * 30)
            
            # Python info
            print(f"🐍 Python: {sys.version}")
            print(f"📁 Working dir: {os.getcwd()}")
            print(f"🛤️ Python path includes GitHub modules: {MODULE_PATH in sys.path}")
            
            # System packages
            packages = ["wget", "curl", "git", "node", "npm"]
            print(f"\n📦 System packages:")
            for pkg in packages:
                try:
                    result = subprocess.run(["which", pkg], capture_output=True)
                    status = "✅" if result.returncode == 0 else "❌"
                    print(f"  {status} {pkg}")
                except:
                    print(f"  ❌ {pkg}")
            
            # Python packages
            print(f"\n🐍 Python packages:")
            py_packages = ["requests", "psutil", "pyngrok", "ipywidgets"]
            for pkg in py_packages:
                try:
                    __import__(pkg)
                    print(f"  ✅ {pkg}")
                except ImportError:
                    print(f"  ❌ {pkg}")
            
            # GitHub modules
            print(f"\n📁 GitHub modules:")
            github_modules = [
                "pinokio_cloud_main", "pinokio_controller", 
                "pinokio_installer", "platform_detector", "tunnel_manager"
            ]
            for module in github_modules:
                try:
                    importlib.import_module(module)
                    print(f"  ✅ {module}")
                except ImportError as e:
                    print(f"  ❌ {module}: {e}")
    
    def check_processes():
        """Check running processes"""
        with debug_output:
            print("🔍 PROCESS CHECK")  
            print("=" * 30)
            
            processes = ["pinokio", "ngrok", "cloudflared", "node"]
            for proc in processes:
                try:
                    result = subprocess.run(
                        ["pgrep", "-f", proc], 
                        capture_output=True, text=True
                    )
                    if result.returncode == 0:
                        pids = result.stdout.strip().split('\n')
                        print(f"  ✅ {proc}: {len(pids)} process(es) - PIDs: {', '.join(pids)}")
                    else:
                        print(f"  ❌ {proc}: Not running")
                except:
                    print(f"  ❌ {proc}: Check failed")
    
    def check_ports():
        """Check port availability"""
        with debug_output:
            print("🔍 PORT CHECK")
            print("=" * 30)
            
            ports = [42000, 4040, 8080, 7860, 3000]
            for port in ports:
                try:
                    result = subprocess.run(
                        ["netstat", "-tlpn"], 
                        capture_output=True, text=True
                    )
                    if f":{port}" in result.stdout:
                        print(f"  🔴 Port {port}: In use")
                    else:
                        print(f"  ✅ Port {port}: Available")
                except:
                    print(f"  ❓ Port {port}: Check failed")
    
    def clear_debug_logs():
        """Clear debug output"""
        debug_output.clear_output()
    
    # Bind events
    check_env_btn.on_click(lambda x: check_environment())
    check_processes_btn.on_click(lambda x: check_processes())
    check_ports_btn.on_click(lambda x: check_ports())
    clear_logs_btn.on_click(lambda x: clear_debug_logs())
    
    # Create UI
    debug_ui = widgets.VBox([
        widgets.HTML("<h3>🔧 Debug & Troubleshooting Tools</h3>"),
        widgets.HBox([
            check_env_btn, check_processes_btn, 
            check_ports_btn, clear_logs_btn
        ]),
        debug_output
    ])
    
    return debug_ui

# Create and display debug tools
debug_tools = create_debug_tools()
display(debug_tools)

In [ ]:
#@title 💾 **Export Configuration & Logs**
import json
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

def create_export_tools():
    """Create tools for exporting configuration and logs"""
    
    export_output = widgets.Output()
    
    export_config_btn = widgets.Button(
        description="💾 Export Config",
        button_style='success'
    )
    
    export_logs_btn = widgets.Button(
        description="📋 Export Logs", 
        button_style='info'
    )
    
    def export_config():
        """Export current configuration"""
        with export_output:
            if pinokio_instance:
                try:
                    config_data = {
                        "timestamp": datetime.now().isoformat(),
                        "status": pinokio_instance.get_status(),
                        "config": pinokio_instance.config,
                        "platform": pinokio_instance.platform,
                        "paths": pinokio_instance.paths
                    }
                    
                    filename = f"pinokio_config_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                    
                    with open(filename, 'w') as f:
                        json.dump(config_data, f, indent=2, default=str)
                    
                    print(f"✅ Configuration exported to: {filename}")
                    
                    # Show first few lines of config
                    print("\n📄 Config preview:")
                    print(json.dumps(config_data, indent=2, default=str)[:500] + "...")
                    
                except Exception as e:
                    print(f"❌ Export failed: {e}")
            else:
                print("❌ No Pinokio instance to export")
    
    def export_logs():
        """Export system logs"""
        with export_output:
            try:
                log_data = {
                    "timestamp": datetime.now().isoformat(),
                    "environment": {
                        "python_version": sys.version,
                        "working_directory": os.getcwd(),
                        "module_path": MODULE_PATH,
                        "repo_path": REPO_DIR
                    },
                    "ui_logs": log_output if 'log_output' in globals() else []
                }
                
                filename = f"pinokio_logs_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                
                with open(filename, 'w') as f:
                    json.dump(log_data, f, indent=2, default=str)
                
                print(f"✅ Logs exported to: {filename}")
                
            except Exception as e:
                print(f"❌ Log export failed: {e}")
    
    # Bind events
    export_config_btn.on_click(lambda x: export_config())
    export_logs_btn.on_click(lambda x: export_logs())
    
    # Create UI
    export_ui = widgets.VBox([
        widgets.HTML("<h3>💾 Export & Backup Tools</h3>"),
        widgets.HBox([export_config_btn, export_logs_btn]),
        export_output
    ])
    
    return export_ui

# Create and display export tools
export_tools = create_export_tools()
display(export_tools)

print("\n🎉 COMPLETE PINOKIO NOTEBOOK READY!")
print("=" * 50)
print("✅ All cells loaded successfully")
print("📁 Using real GitHub implementation")
print("🎛️ Advanced UI controls available")
print("🔧 Debug tools included")
print("💾 Export tools ready")
print("\n🚀 Use the UI controls above to deploy Pinokio!")